## Datashading a 2.7-billion-point Open Street Map database

Most [datashader](https://github.com/bokeh/datashader) examples use "medium-sized" datasets, because they need to be small enough to be distributed over the internet without racking up huge bandwidth charges for the project maintainers.  But given that Datashader supports [Dask](http://dask.pydata.org) dataframes, Datashader can easily be used with truly large datasets.  Such datasets can be much larger than the available amount of physical memory on the machine, either by paging in data on a single machine, or by distributing data across multiple machines. Here we illustrate how to work "out of core" on a single machine.

This example will use a 2.7-billion point dataset, which is unfortunately too large to distribute with Datashader (7GB compressed). The data is taken from Open Street Map's (OSM) [bulk GPS point data](https://blog.openstreetmap.org/2012/04/01/bulk-gps-point-data/). The data was collected by OSM contributors' GPS devices, and was provided as a CSV file of `latitude,longitude` coordinates. The data was downloaded from their website, extracted, converted to use positions in Web Mercator format using `datashader.utils.lnglat_to_meters()`, and then stored in a [parquet](https://github.com/dask/fastparquet) file for [faster disk access](https://github.com/bokeh/datashader/issues/129#issuecomment-300515690). To run this notebook, you would need to do the same process yourself to obtain `osm.snappy.parq`.   Once you have it, you can follow the steps below to load and plot the data.

In [ ]:
import dask.dataframe as dd
import dask.diagnostics as diag
import datashader as ds
import datashader.transfer_functions as tf

from functools import partial
from colorcet import fire
from datashader.utils import export_image
from datashader.colors import colormap_select

In [ ]:
df = dd.io.parquet.read_parquet('data/osm.snappy.parq')
df.head()

### Aggregation

First, we create a canvas to provide pixel-shaped bins in which points can be aggregated, and then aggregate the data to produce a fixed-size aggregate array.

In [ ]:
bound = 20026376.39
bounds = dict(x_range = (-bound, bound), y_range = (int(-bound*0.4), int(bound*0.6)))
plot_width = 1000
plot_height = int(plot_width*0.5)

cvs = ds.Canvas(plot_width=plot_width, plot_height=plot_height, **bounds)

with diag.ProgressBar(), diag.Profiler() as prof, diag.ResourceProfiler(0.5) as rprof:
    agg = cvs.points(df, 'x', 'y', ds.count())

### Transfer Function

Next, we create an image out of the aggregate array, by mapping small (but nonzero) counts to light blue, the largest counts to dark blue, and interpolating according to a logarithmic function in between.

In [ ]:
tf.shade(agg, cmap=["lightblue", "darkblue"], how='log')

There's some odd, low-count, nearly-uniform noise going on in the tropics. It's worth trying to figure out what that could be, but for now we can filter it out quickly from the aggregated data using the `where` method, and switch to the parameter-free histogram equalization method for mapping to colors using various colormaps:

In [ ]:
tf.shade(agg.where(agg > 15), cmap=["lightblue", "darkblue"])

In [ ]:
export = partial(export_image, export_path="export", background="black")

export(tf.shade(agg.where(agg > 15), cmap=["darkblue", "lightcyan"]), "OSM_black_blue")

In [ ]:
export(tf.shade(agg.where(agg > 15), cmap=fire), "OSM_fire")

The result is a decent map of world population, with Europe apparently having particularly many OpenStreetMap contributors. The long great-circle paths are presumably flight or boat trips, from devices that log their GPS coordinates more than 15 times during the space of one pixel in this plot (or else they would have been elimnated by the `where` call).

### Performance Profile

Dask offers some tools to visualize how memory and processing power are being used during these calculations:

In [ ]:
from bokeh.io import output_notebook
from bokeh.resources import CDN
output_notebook(CDN, hide_banner=True)

In [ ]:
diag.visualize([prof, rprof])
None

Performance notes:
- On a 16GB machine, most of the time is spent reading the data from disk (the purple rectangles)
- Reading time includes not just disk I/O, but decompressing chunks of data
- The disk reads don't release the [Global Interpreter Lock](https://wiki.python.org/moin/GlobalInterpreterLock) (GIL), and so CPU usage (see second chart above) drops to only one core during those periods.
- During the aggregation steps (the green rectangles), CPU usage on this machine with 8 hyperthreaded cores (4 full cores) spikes to nearly 800%, because the aggregation function is implemented in parallel. 
- The data takes up 11 GB of memory when uncompressed, but only a peak of around 6 GB of physical memory is ever used because the data is paged in as needed.

### Interactive plotting

If you have enough RAM to hold the whole dataset (or are very patient), you can uncomment the `InteractiveImage` line below and run the cell to build an interactive plot where you can select a region for zooming. Without enough RAM, computation has to be done out of core, and it could take a half-minute or so to process a pan or zoom event before the plot gets updated.

In [ ]:
from bokeh.plotting import figure, output_notebook
from bokeh.io import push_notebook
from datashader.bokeh_ext import InteractiveImage
from datashader import transfer_functions as tf

def create_image(x_range, y_range, w, h):
    cvs = ds.Canvas(x_range=x_range, y_range=y_range)
    agg = cvs.points(df, 'x', 'y', ds.count())
    return tf.shade(agg.where(agg > 20), cmap=["lightcyan", "darkblue"], how="log")

p = figure(tools='pan,wheel_zoom,box_zoom,reset', plot_width=plot_width, plot_height=plot_height, **bounds)
           
p.axis.visible = False
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#InteractiveImage(p, create_image)